In [ ]:
pip install openpyxl

In [ ]:
%pip install webdriver-manager

In [1]:
import time
from openpyxl import Workbook
import pandas as pd
from bs4 import BeautifulSoup
#
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#chrome driver 링크 연결 없이 바로 하는 법
#%pip install webdriver-manager (install받아야함)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

#경고메세지 지워줌
import warnings 
warnings.filterwarnings('ignore')

In [38]:
#엑셀 시트 생성
wb = Workbook(write_only=True)
ws = wb.create_sheet()


In [26]:
#크롬드라이버 링크 연결
# path='C:\chromedriver_win32\chromedriver.exe'
# driver= webdriver.Chrome(path) ->
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options())

#스크롤 시 풀 스크린으로 켜지게되면 우측에 '추천영상' 때문에 댓글보다 더 스크롤이 내려가게 됨
#하프 사이즈의 스크린으로 켜게 되면 추천영상이 일정이상 새로고침되지 않음 
driver.set_window_size(800, 1100)
#이 상태에서는 링크창에 data;,라고 적힌채로 대기상태에 들어감

In [27]:
#1. 검색어 입력받기
search=input()
driver.implicitly_wait(3)


#2. 유튜브 실행
driver.get("https://www.youtube.com")
#3초안에 웹페이지를 로딩하면 바로넘어가거나, 3초를 기다림
driver.implicitly_wait(3)

#get(링크)로 안하고 이것만 실행하면
#현재 페이지에 합쳐진다!!!

######검색#######


#3. 검색어 send
#ID,CLASS_NAME으로 하면 send가 안되거나 click조차 안될때가 있었음
#NAME으로 하면 둘 다 해결됨 이유는 모름?
search_text=driver.find_element(By.NAME,'search_query')

#로딩이 다 안됐기때문에 1초 기다려줌
driver.implicitly_wait(1)
time.sleep(1)

#4. 검색어 전송
search_text.send_keys(search+" -Shorts")
driver.implicitly_wait(5)

#검색버튼 누르기
# search_btn=driver.find_element(By.CLASS_NAME,'style-scope.ytd-searchbox')
# search_btn.click()
# driver.implicitly_wait(3)

#5. 검색(엔터 누르기)
search_text.send_keys(Keys.ENTER)

In [ ]:
######영상 선정######
id='dismissible'



In [29]:
######스크롤하기######
#댓글 전체 스크랩하기 위해

#275개의 댓글 >> 23.5s , 23.2s , 24.0s 23~24s


#화면 최하단으로 스크롤 이동 : scrollTo(0, document.body.scrollHeight)
#자바 스크립트 문법 >execute_script(자바스크립트를 실행) > scrollTo

#window.scrollTo(x-좌표, y-좌표)
#시작위치지정(댓글창이 시작되는 위치)
#화면사이즈를 반으로 줄이고나니 스크롤을 끝까지 내린 후에 댓글이 생김
driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
driver.implicitly_wait(10)

#스크롤 이전 높이
#화면 바깥으로 삐져나간 부분까지 포함해서 전체 글의 길이를 scrollHeight
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    #스크롤의 y좌표를 가장아래(scrollHeight)까지 내림
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.5)

    #스크롤 후 높이 구하기
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    #끝까지 스크롤 한 뒤 멈추기
    if new_height == last_height:
        break
    last_height = new_height

driver.implicitly_wait(1.5)


In [ ]:
######유튜브 팝업 닫기######

#find_element_by_css_selector는 과거의 속성 find_element를 쓰는게 나음
try:
    driver.find_element(By.CSS_SELECTOR,"#dismiss-button > a").click()
except:
    pass

In [30]:
######대댓글 토글 열기######
# replies = driver.find_element(By.CSS_SELECTOR,"#more-replies > a")
# replies = driver.find_elements(By.CSS_SELECTOR,"#more-replies>a")

#find_elements < 무려 s를 안해서틀렸었음 하하 
replies = driver.find_elements(By.CLASS_NAME,"more-button")

time.sleep(1)

for reply in replies:
    #click도 안됨 send_keys도 안됨
    driver.execute_script("arguments[0].click();", reply)
    time.sleep(1.5) 

# reply.click()
# reply.send_keys(Keys.ENTER)
# 이거 안됨 스크립트로 해결함 근데 11분 18초 걸림!!!!!

In [37]:
######댓글 가져오기#######
#page_source
#브라우저에 보이는 그대로의 HTML, 크롬 개발자 도구의 Element 탭 내용과 동일.
html_source = driver.page_source
#BeautifulSoup(html문서,'html.parser')
soup = BeautifulSoup(html_source, 'html.parser')

#soup.select 조건에 맞는 태그를 여러개 가져옴 
#soup.select_one() 조건에 맞는 태그를 한개(여러개가 있어도 한개만)
#태그이름  .클래스이름  #아이디이름  상위태그>자식태그>자식태그
#아이디>태그.클래스이름
id_list = soup.select("div#header-author > h3 > #author-text > span")
comment_list = soup.select("yt-formatted-string#content-text")

id_final = []
comment_final = []

for i in range(len(comment_list)):
    temp_id = id_list[i].text  #아이디 리스트를 가져와서
    temp_id = temp_id.replace('\n', '') #필요한 부분만 남기면
    temp_id = temp_id.replace('\t', '')
    temp_id = temp_id.replace('    ', '')
    id_final.append(temp_id) # 댓글 작성자 id_final이 생김

    temp_comment = comment_list[i].text #마찬가지
    temp_comment = temp_comment.replace('\n', '')
    temp_comment = temp_comment.replace('\t', '')
    temp_comment = temp_comment.replace('    ', '')
    comment_final.append(temp_comment) # 댓글 내용

# #결과확인
# print("아이디 출력: ")
# print(id_final)
# print('=================')
# print("댓글 출력: ")
# print(comment_final)

In [39]:
######추출한 아이디 댓글 저장하기######
pd_data = {"아이디" : id_final , "댓글 내용" : comment_final}
#pandas DataFrame만들기 =Table형식의 데이터를 다룰 때 사용한다.
youtube_pd = pd.DataFrame(pd_data)

youtube_pd.to_excel('result.xlsx')